# 🧱 Cell 1：环境与路径设置（GPU6）

In [15]:
# Cell 1: 环境与路径设置（GPU6）

import os

# 只使用 GPU6
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

# 设置项目根目录（按你实际路径来）
PROJECT_ROOT = "/beacon/data01/chengjie.zheng001/Projects/07TRACE_LLMs/26ACL/TRACE_26ACL"

# 确保当前工作目录是项目根目录
os.chdir(PROJECT_ROOT)

print("Current working dir:", os.getcwd())
print("CUDA_VISIBLE_DEVICES:", os.environ.get("CUDA_VISIBLE_DEVICES"))


Current working dir: /beacon/data01/chengjie.zheng001/Projects/07TRACE_LLMs/26ACL/TRACE_26ACL
CUDA_VISIBLE_DEVICES: 6


# 📦 Cell 2：安装 / 导入依赖（如果已经装好可以跳过安装行）

In [16]:
# Cell 2: 导入依赖（如果缺包再 pip 安装）

import sys
import json
from typing import List, Dict, Any

import numpy as np
import torch
from tqdm import tqdm

try:
    import transformers
    from transformers import AutoTokenizer, AutoModelForCausalLM
except ImportError:
    # 如果报错就取消下一行注释
    # !pip install transformers accelerate
    raise

try:
    import wandb
    WANDB_AVAILABLE = True
except ImportError:
    WANDB_AVAILABLE = False
    print("wandb not installed, set USE_WANDB = False later.")


# 📚 Cell 3：加载 LogicNLI 数据集

In [17]:
# Cell 3: 加载 LogicNLI 数据集

import os

def load_json_or_jsonl(path: str) -> List[Dict[str, Any]]:
    if not os.path.exists(path):
        raise FileNotFoundError(f"File not found: {path}")

    if path.endswith(".jsonl"):
        data = []
        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                data.append(json.loads(line))
        return data

    with open(path, "r", encoding="utf-8") as f:
        obj = json.load(f)

    if isinstance(obj, list):
        return obj
    if isinstance(obj, dict):
        if "data" in obj and isinstance(obj["data"], list):
            return obj["data"]
        return [obj]
    raise ValueError(f"Unsupported JSON format: {path}")


def load_logicnli(split: str = "dev") -> List[Dict[str, Any]]:
    path = os.path.join(PROJECT_ROOT, "data", "LogicNLI", f"{split}.json")
    print(f"[INFO] Loading LogicNLI from {path}")
    data = load_json_or_jsonl(path)
    print(f"[INFO] Loaded {len(data)} examples.")
    return data


logicnli_data = load_logicnli("dev")

# 简单看一条样本的字段，方便你确认格式
print("Sample keys:", list(logicnli_data[0].keys()))
print("Sample example:", logicnli_data[0])


[INFO] Loading LogicNLI from /beacon/data01/chengjie.zheng001/Projects/07TRACE_LLMs/26ACL/TRACE_26ACL/data/LogicNLI/dev.json
[INFO] Loaded 300 examples.
Sample keys: ['id', 'facts', 'rules', 'conjecture', 'answer']
Sample example: {'id': 'LogicNLI_1', 'facts': ['Vivian is eager.', 'Rhett is not modest.', 'Vivian is confused.', 'Vivian is not lazy.', 'Atwater is eager.', 'Philip is not confused.', 'Giles is not eager.', 'Giles is not real.', 'Kilby is not eager.', 'Ramon is eager.', 'Rhett is not sociable.', 'Rhett is lazy.'], 'rules': ['Someone being not eager is equivalent to being not sociable.', 'If someone is not lazy, then he is both sociable and modest.', 'If there is someone who is not eager, then Ramon is not lazy.', 'Philip being eager or Kilby being real implies that Philip is not lazy.', 'If there is someone who is both not sociable and not modest, then Kilby is not lazy.', 'If someone is modest or not confused, then he is not eager.', 'It can be concluded that Atwater is ea

# ✏️ Cell 4：构造输入文本（LogicNLI → LLM prompt）

In [18]:
# Cell 4: 构造 LogicNLI 的 LLM 输入文本
# 按常见 LogicNLI 格式写了一个版本，你可以根据上一 cell 打印的字段微调

def build_input_text_logicnli(ex: Dict[str, Any]) -> str:
    """
    假设 LogicNLI 每条样本大概有：
        - premise / context
        - hypothesis / query
        - label (entailment / contradiction / neutral) 之类
    你可以根据实际字段名做替换。
    """
    premise = ex.get("premise", ex.get("context", ""))
    hypothesis = ex.get("hypothesis", ex.get("query", ""))

    # 也可以把 label 信息去掉，避免泄漏答案
    # label = ex.get("label", "")

    text = (
        "You are a logical reasoner. "
        "Given the premise and hypothesis, you analyze their logical relation.\n\n"
        f"Premise: {premise}\n"
        f"Hypothesis: {hypothesis}\n"
    )
    return text


# 简单测试构造的 prompt
print(build_input_text_logicnli(logicnli_data[0]))


You are a logical reasoner. Given the premise and hypothesis, you analyze their logical relation.

Premise: 
Hypothesis: 



# 🧠 Cell 5：在 notebook 里定义 LLaMA-3 后端（HFTraceCausalLM）

In [27]:
# Cell 5: 定义 HFTraceCausalLM（去掉 device_map，不依赖 accelerate）

class HFTraceCausalLM:
    def __init__(
        self,
        model_name: str = "TinyLlama/TinyLlama-1.1B-Chat-v1.0",  # 先用 TinyLlama debug
        device: str = "cuda",
        dtype: str = "bfloat16",
        max_length: int = 1024,
    ):
        self.model_name = model_name
        self.device = device
        self.max_length = max_length

        print(f"[INFO] Loading tokenizer: {model_name}")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        # dtype 映射
        dtype_map = {
            "float32": torch.float32,
            "float16": torch.float16,
            "bfloat16": torch.bfloat16,
        }
        torch_dtype = dtype_map[dtype]

        print(f"[INFO] Loading model: {model_name} with dtype={dtype}")
        # 关键改动：不再传 device_map，这样就不需要 accelerate
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch_dtype,
        )
        # 手动把模型搬到指定设备（这里就是 GPU6，因为你设了 CUDA_VISIBLE_DEVICES=6）
        self.model.to(self.device)
        self.model.eval()

    @torch.no_grad()
    def encode(self, prompts: List[str]) -> Dict[str, Any]:
        batch_enc = self.tokenizer(
            prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=self.max_length,
        )

        # 把输入搬到模型所在设备
        batch_enc = {k: v.to(self.device) for k, v in batch_enc.items()}

        outputs = self.model(
            **batch_enc,
            output_hidden_states=True,
            use_cache=False,
        )

        hidden_states = outputs.hidden_states  # tuple(num_layers+1, B, T, D)
        # 这里直接在 CPU 上转成 float32，后面任何地方用到 hidden_states 都不会再遇到 bfloat16 的问题
        hs_list = [h.detach().to(torch.float32).cpu() for h in hidden_states]

        tokens = [
            self.tokenizer.convert_ids_to_tokens(ids.tolist())
            for ids in batch_enc["input_ids"]
        ]

        return {
            "hidden_states": hs_list,
            "tokens": tokens,
            "model_name": self.model_name,
            "meta": {
                "num_layers": len(hs_list) - 1,
                "seq_lens": batch_enc["attention_mask"].sum(dim=-1).tolist(),
            },
        }


# 测试一下模型能否正常 forward 一小条
llama_model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # 先用这个，之后再换回 LLaMA-3
hf_model = HFTraceCausalLM(
    model_name=llama_model_name,
    device="cuda",         # 对应 CUDA_VISIBLE_DEVICES=6
    dtype="bfloat16",
    max_length=512,
)

test_prompt = build_input_text_logicnli(logicnli_data[0])
test_outputs = hf_model.encode([test_prompt])

print("num hidden states (layers+embedding):", len(test_outputs["hidden_states"]))
print("tokens of first example (truncated):", test_outputs["tokens"][0][:20])



[INFO] Loading tokenizer: TinyLlama/TinyLlama-1.1B-Chat-v1.0
[INFO] Loading model: TinyLlama/TinyLlama-1.1B-Chat-v1.0 with dtype=bfloat16
num hidden states (layers+embedding): 23
tokens of first example (truncated): ['<s>', '▁You', '▁are', '▁a', '▁logical', '▁reason', 'er', '.', '▁Given', '▁the', '▁prem', 'ise', '▁and', '▁hypothesis', ',', '▁you', '▁analyze', '▁their', '▁logical', '▁relation']


# 💾 Cell 6：保存每层 embedding 的工具函数

In [28]:
# Cell 6: 保存每层 embedding 的工具函数

def save_embeddings(
    per_layer_vecs: List[np.ndarray],
    ex_id: str,
    model_name: str,
    dataset: str,
    save_root: str,
):
    """
    per_layer_vecs: List[np.ndarray]，每个 (D,) 或 (T, D)
    ex_id: 样本 id（string）
    """
    model_short = model_name.split("/")[-1].replace(":", "_")
    save_dir = os.path.join(save_root, dataset, model_short)
    os.makedirs(save_dir, exist_ok=True)

    save_path = os.path.join(save_dir, f"ex_{ex_id}.npz")
    np.savez_compressed(save_path, *per_layer_vecs)
    return save_path


EMBED_SAVE_ROOT = os.path.join(PROJECT_ROOT, "results", "embeddings")
print("Embedding will be saved under:", EMBED_SAVE_ROOT)


Embedding will be saved under: /beacon/data01/chengjie.zheng001/Projects/07TRACE_LLMs/26ACL/TRACE_26ACL/results/embeddings


# 📈 Cell 7：配置 wandb（可选）

In [29]:
# Cell 7: 配置 wandb（可选）

USE_WANDB = True and WANDB_AVAILABLE  # 想关掉就改成 False

WANDB_PROJECT = "TRACE_26ACL"
WANDB_ENTITY = None  # 如果你有 entity，就填你的 entity 字符串；否则 None

run_name = f"LogicNLI_dev_{llama_model_name.split('/')[-1]}_layerwise"

if USE_WANDB:
    print("[INFO] Initializing wandb...")
    wandb.login()  # 如果之前没登录过，会提示你粘 token
    wandb.init(
        project=WANDB_PROJECT,
        entity=WANDB_ENTITY,
        name=run_name,
        config={
            "dataset": "LogicNLI",
            "split": "dev",
            "model_name": llama_model_name,
            "max_length": 512,
            "note": "layerwise embeddings from notebook on GPU6",
        },
    )
else:
    print("[INFO] wandb disabled.")


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


[INFO] Initializing wandb...


# 🚀 Cell 8：主循环——抽取每层 embedding & 保存

In [30]:
# Cell 8: 主循环——抽取每层 embedding & 保存

BATCH_SIZE = 4         # 可以根据 GPU 显存改大/改小
MAX_SAMPLES = -1       # >0 则只跑前 N 条样本，便于 debug
DATASET_NAME = "LogicNLI"
SPLIT = "dev"

data = logicnli_data
if MAX_SAMPLES > 0:
    data = data[:MAX_SAMPLES]
    print(f"[INFO] Using first {len(data)} samples (MAX_SAMPLES={MAX_SAMPLES}).")

num_examples = len(data)
print(f"[INFO] Total examples to process: {num_examples}")

layer_norm_sums = None
layer_norm_counts = 0
num_layers_plus_one = None

global_idx = 0

for start in tqdm(range(0, num_examples, BATCH_SIZE), desc="Batches"):
    end = min(start + BATCH_SIZE, num_examples)
    batch = data[start:end]

    prompts = [build_input_text_logicnli(ex) for ex in batch]

    outputs = hf_model.encode(prompts)
    hidden_states = outputs["hidden_states"]   # list[L+1]，每个 (B, T, D)
    model_name = outputs.get("model_name", llama_model_name)

    if num_layers_plus_one is None:
        num_layers_plus_one = len(hidden_states)
        print(f"[INFO] Model returned {num_layers_plus_one} hidden states (incl. embedding layer).")

    for b, ex in enumerate(batch):
        ex_id = str(ex.get("id", global_idx))   # 如果样本没有 id 字段，就用 index

        per_layer_vecs = []
        layer_norms = []

        for layer_idx, h in enumerate(hidden_states):
            # h: (B, T, D) torch.Tensor
            seq_hidden = h[b]          # (T, D)

            # 这里我们取最后一个 token 的向量作为该层的 embedding
            last_vec = seq_hidden[-1]  # (D,)
            vec_np = last_vec.detach().cpu().numpy()
            per_layer_vecs.append(vec_np)

            layer_norms.append(float(last_vec.norm(p=2).item()))

        save_path = save_embeddings(
            per_layer_vecs=per_layer_vecs,
            ex_id=ex_id,
            model_name=model_name,
            dataset=DATASET_NAME,
            save_root=EMBED_SAVE_ROOT,
        )

        global_idx += 1

        if global_idx <= 3:
            print(f"Example {global_idx} saved to {save_path}")

        # 更新全局统计
        if layer_norm_sums is None:
            layer_norm_sums = np.zeros(len(layer_norms), dtype=np.float64)
        layer_norm_sums += np.array(layer_norms, dtype=np.float64)
        layer_norm_counts += 1

    # 每个 batch 之后，用 wandb 记录一下各层平均 L2 范数
    if USE_WANDB:
        avg_layer_norms = layer_norm_sums / max(layer_norm_counts, 1)
        log_dict = {
            f"layer_{i}_avg_l2": float(v)
            for i, v in enumerate(avg_layer_norms)
        }
        log_dict["num_processed"] = layer_norm_counts
        wandb.log(log_dict)

print(f"[INFO] Done. Total processed examples: {layer_norm_counts}")

if USE_WANDB:
    wandb.finish()


[INFO] Total examples to process: 300


Batches:   1%|██▏                                                                                                                                                                | 1/75 [00:00<00:11,  6.23it/s]

[INFO] Model returned 23 hidden states (incl. embedding layer).
Example 1 saved to /beacon/data01/chengjie.zheng001/Projects/07TRACE_LLMs/26ACL/TRACE_26ACL/results/embeddings/LogicNLI/TinyLlama-1.1B-Chat-v1.0/ex_LogicNLI_1.npz
Example 2 saved to /beacon/data01/chengjie.zheng001/Projects/07TRACE_LLMs/26ACL/TRACE_26ACL/results/embeddings/LogicNLI/TinyLlama-1.1B-Chat-v1.0/ex_LogicNLI_2.npz
Example 3 saved to /beacon/data01/chengjie.zheng001/Projects/07TRACE_LLMs/26ACL/TRACE_26ACL/results/embeddings/LogicNLI/TinyLlama-1.1B-Chat-v1.0/ex_LogicNLI_3.npz


Batches: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 75/75 [00:06<00:00, 11.77it/s]

[INFO] Done. Total processed examples: 300


layer_0_avg_l2,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
layer_10_avg_l2,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
layer_11_avg_l2,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
layer_12_avg_l2,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
layer_13_avg_l2,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
layer_14_avg_l2,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
layer_15_avg_l2,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
layer_16_avg_l2,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
layer_17_avg_l2,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
layer_18_avg_l2,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
+14,...


# 🔍 Cell 9：简单检查保存的 embedding

In [31]:
# Cell 9: 检查一下保存的 embedding 文件结构

import glob

model_short = llama_model_name.split("/")[-1].replace(":", "_")
embed_dir = os.path.join(EMBED_SAVE_ROOT, DATASET_NAME, model_short)
print("Embedding dir:", embed_dir)

files = sorted(glob.glob(os.path.join(embed_dir, "ex_*.npz")))
print("Num files:", len(files))
print("First 3 files:", files[:3])

# 看一眼一个 .npz 里有几层，以及向量维度
if files:
    sample_npz = np.load(files[0])
    print("Num layers stored:", len(sample_npz.files))
    for i, key in enumerate(sample_npz.files):
        arr = sample_npz[key]
        print(f"layer {i}: shape {arr.shape}")
        if i >= 3:
            break


Embedding dir: /beacon/data01/chengjie.zheng001/Projects/07TRACE_LLMs/26ACL/TRACE_26ACL/results/embeddings/LogicNLI/TinyLlama-1.1B-Chat-v1.0
Num files: 300
First 3 files: ['/beacon/data01/chengjie.zheng001/Projects/07TRACE_LLMs/26ACL/TRACE_26ACL/results/embeddings/LogicNLI/TinyLlama-1.1B-Chat-v1.0/ex_LogicNLI_1.npz', '/beacon/data01/chengjie.zheng001/Projects/07TRACE_LLMs/26ACL/TRACE_26ACL/results/embeddings/LogicNLI/TinyLlama-1.1B-Chat-v1.0/ex_LogicNLI_10.npz', '/beacon/data01/chengjie.zheng001/Projects/07TRACE_LLMs/26ACL/TRACE_26ACL/results/embeddings/LogicNLI/TinyLlama-1.1B-Chat-v1.0/ex_LogicNLI_100.npz']
Num layers stored: 23
layer 0: shape (2048,)
layer 1: shape (2048,)
layer 2: shape (2048,)
layer 3: shape (2048,)
